# Deep Learning - Introduction à Pytorch 


## TP2 : Fonctions Dérivables

Sylvain Lamprier (sylvain.lamprier@univ-angers.fr)

Supports adaptés de Nicolas Baskiotis (nicolas.baskiotis@sorbonne-univeriste.fr) et Benjamin Piwowarski (benjamin.piwowarski@sorbonne-universite.fr) -- MLIA/ISIR, Sorbonne Université

In [1]:
import torch
print("La version de torch est : ",torch.__version__)
print("Le calcul GPU est disponible ? ", torch.cuda.is_available())

import numpy as np
import sklearn

La version de torch est :  1.12.1
Le calcul GPU est disponible ?  False


Au TP précédent, nous avons vu comment implémenter une regression linéaire en utilisant les structures Tensor de PyTorch. Cependant, nous exploitions pas du tout la puissance de PyTorch qui permet de faciliter le calcul des gradients via de l'auto-dérivation. Dans le TP précédent nous avions défini un algorithme spécifique à de la regression pour un modèle (linéaire) et un coût (moindres carrés) figés, en définissant à la main le gradient du coût global pour l'ensemble des paramètres. Ce mode de programmation est très peu modulaire et est très difficilement étendable à des architectures plus complexes. Sachant que l'objectif est de développer des architectures neuronales avec des nombreux modules neuronaux enchaînés, il n'est pas possible de travailler de cette façon. 

Dans ce TP, nous allons voir comment décomposer les choses pour rendre le code plus facilement généralisable. L'objectif est de comprendre le fonctionnement interne de PyTorch (sans en utiliser encore les facilités offertes par l'utilisation d'un graphe de calcul), basé sur l'implémentation d'objets Function.  

## Fonctions


$\href{https://pytorch.org/docs/stable/}{\texttt{PyTorch}}$ utilise une classe abstraite $\href{https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function}{\texttt{Function}}$ dont sont héritées toutes les fonctions et qui nécessite l'implémentation de ces deux méthodes :

- méthode $\texttt{forward(ctx, *inputs)}$ : calcule le résultat de l'application de la fonction
- méthode $\texttt{backward(ctx, *grad-outputs)}$ : calcule le gradient partiel par rapport à chaque entrée de la méthode $\texttt{forward}$; le nombre de $\texttt{grad-outputs}$ doit être égale aux nombre de sorties de $\texttt{forward}$ (pourquoi ?) et le nombre de  
sorties doit être égale aux nombres de $\texttt{inputs}$ de $\texttt{forward}$.


Pour des raisons d'implémentation, les deux méthodes doivent être statiques. Le premier paramètre $\texttt{ctx}$ permet de sauvegarder un contexte lors de la passe $\texttt{forward}$ (par exemple les tenseurs d'entrées) et il est passé lors de la passe $\texttt{backward}$ en paramètre afin de récupérer les valeurs. $\textbf{Attention : }$ le contexte doit être unique pour chaque appel de $\texttt{forward}$.

Compléter le code ci-dessous pour créer des modules MSE (coût moindres carrés) et Linéaire. Les deux cellules en dessous vous serviront à tester votre code: si tout se passe sans plantage, alors vos gradients semblent corrects. Utiliser bien les outils propres à pyTorch, en particulier des Tensor et pas des matrices numpy. Assurez vous que
vos fonctions prennent en entrée des batchs d’exemples (matrice 2D) et non un seul exemple (vecteur). N’hésiter pas à prendre un exemple et déterminer les dimensions des différentes matrices en jeu.  

In [2]:
import torch
from torch.autograd import Function
from torch.autograd import gradcheck


class Context:
    """Un objet contexte très simplifié pour simuler PyTorch
    
    Un contexte différent doit être utilisé à chaque forward
    """
    def __init__(self):
        self._saved_tensors = ()
    def save_for_backward(self, *args):
        self._saved_tensors = args
    @property
    def saved_tensors(self):
        return self._saved_tensors


class MSE(Function):
    """Début d'implementation de la fonction MSE"""
    @staticmethod
    def forward(ctx, yhat, y):
        ## Garde les valeurs nécessaires pour le backwards
        ctx.save_for_backward(yhat, y)

        # [[STUDENT]] Renvoyer la valeur de la fonction
        
        loss = torch.mean((yhat - y)**2)
        return loss
    
        # [[/STUDENT]]

    @staticmethod
    def backward(ctx, grad_output):
        ## Calcul du gradient du module par rapport à chaque groupe d'entrées
        yhat, y = ctx.saved_tensors

        # Calcul des dérivées partielles
        grad_yhat = 2 * (yhat - y) / yhat.numel()
        grad_y = -2 * (yhat - y) / yhat.numel()

        return grad_yhat * grad_output, grad_y * grad_output
        
        # [[/STUDENT]]

# [[STUDENT]] Implémenter la fonction Linear(X, W, b)sur le même modèle que MSE

class Linear(Function):
    """Implémentation de la fonction Linear"""
    @staticmethod
    def forward(ctx, X, W, b):
        ctx.save_for_backward(X, W, b)
        output = torch.matmul(X, W) + b

        return output

    @staticmethod
    def backward(ctx, grad_output):
        X, W, b = ctx.saved_tensors
        
        grad_X = torch.matmul(grad_output, W.t())
        grad_W = torch.matmul(X.t(), grad_output)
        grad_b = torch.sum(grad_output, dim=0)
        return grad_X, grad_W, grad_b

# [[/STUDENT]]


linear = Linear.apply
mse = MSE.apply





In [3]:
# Test du gradient de MSE 
yhat = torch.randn(10,5, requires_grad=True, dtype=torch.float64)
y = torch.randn(10,5, requires_grad=True, dtype=torch.float64)
torch.autograd.gradcheck(mse, (yhat, y))

True

In [4]:
# Test du gradient de Linear (sur le même modèle que MSE)

x = torch.randn(13, 5,requires_grad=True,dtype=torch.float64)
w = torch.randn(5, 7,requires_grad=True,dtype=torch.float64)
b = torch.randn(7,requires_grad=True,dtype=torch.float64)
torch.autograd.gradcheck(linear,(x,w,b))

True

## Descente de Gradient

Compléter ci-dessous le code pour réaliser la même regression linéaire qu'au TP précédent, mais en utilisant les objets Function déclarés ci-dessus. 

In [5]:
## Chargement des données Boston et transformation en tensor.
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()
x = torch.tensor(housing['data'],dtype=torch.float)
y = torch.tensor(housing['target'],dtype=torch.float).view(-1,1)

print("Nombre d'exemples : ",x.size(0), "Dimension : ",x.size(1))
print("Nom des attributs : ", ", ".join(housing['feature_names']))

#initialisation aléatoire de w et b
w = torch.randn(x.size(1),1)
b =  torch.randn(1,1)

EPOCHS = 5000
EPS = 1e-6
for n_iter in range(EPOCHS):
    ## [[STUDENT]] Calcul du forward (loss), avec creation de nouveaux Context pour chaque module
    
    mse_context = Context()
    linear_context = Context()

    yhat = Linear.apply(linear_context, x, w, b)
    loss = MSE.apply(mse_context, yhat, y)
    
    # [[/STUDENT]]

    # `loss` doit correspondre au coût MSE calculé à cette itération
    if n_iter % 100==0: 
        print(f"Itérations {n_iter}: loss {loss}")

    ## [[STUDENT]] Calcul du backward (grad_w, grad_b)
    grad_w, grad_b = torch.autograd.grad(loss, (w, b), retain_graph=True)
    # [[/STUDENT]]

    ## [[STUDENT]] Mise à jour des paramètres du modèle
    
    # [[/STUDENT]]
    with torch.no_grad():
        w -= EPS * grad_w
        b -= EPS * grad_b

Nombre d'exemples :  20640 Dimension :  8
Nom des attributs :  MedInc, HouseAge, AveRooms, AveBedrms, Population, AveOccup, Latitude, Longitude


TypeError: Linear.forward() takes 4 positional arguments but 5 were given